In [16]:
import os
import sys
import subprocess
import re
from glob import glob
import tempfile
import winreg
import xml.etree.ElementTree as ET
#from timeit import default_timer as timer

In [5]:
def tag_iterator(container, tag):
    for element in container.iter():
        if element.tag == tag:
            yield element

def tag_iterator_old(container, tag):
    if container.tag == tag:
        yield container
    else:
        for element in container:
            yield from tag_iterator_old(element, tag)


In [37]:
# Get Arma 3 root from registry
reg = winreg.ConnectRegistry(None, winreg.HKEY_LOCAL_MACHINE)
key = winreg.OpenKey(reg, r'SOFTWARE\Wow6432Node\bohemia interactive\arma 3')
a3_root = winreg.EnumValue(key,1)[1]

tmpdir = tempfile.TemporaryDirectory()

# Unpack A3 language PBOs
language_pbos = glob(os.path.join(a3_root, r'Addons\language_f*.pbo'))
language_pbos += glob(os.path.join(a3_root, r'*\Addons\language_f*.pbo'))
for pbo in language_pbos:
    src_dir = os.path.splitext(os.path.basename(pbo))[0]
    subprocess.check_output(['armake.exe', 'unpack', pbo, os.path.join(tmpdir.name, src_dir)])

# Populate a3_stringtable_dict with A3 localizations
a3_stringtable_dict = {}
for file in glob(os.path.join(tmpdir.name, r'*\stringtable.xml')):
    root = ET.parse(file).getroot()
    for entity in tag_iterator(root, 'Key'):
        loc_id = entity.attrib['ID']
        loc_text = entity.find('English').text
        a3_stringtable_dict[loc_text] = loc_id

# Search for duplicates in ZEN
to_replace = []
for file in glob(r'..\addons\*\stringtable.xml'):
    addon = os.path.dirname(file)
    root = ET.parse(file).getroot()
    for entity in tag_iterator(root, 'Key'):
        try:
            zen_loc_id = entity.attrib['ID']
            zen_loc_text = entity.find('English').text
            a3_loc_id = a3_stringtable_dict[zen_loc_text]
            #print(zen_loc_id, '<=>', a3_loc_id)
            to_replace.append([zen_loc_id, a3_loc_id])
        except KeyError:
            continue

addons = list(map(os.path.basename, glob(r'..\addons\*')))

for entry in to_replace:
    for addon in addons:
        to_match = '_{}_'.format(addon)
        match = re.search(to_match, entry[0], re.IGNORECASE)
        if match is not None:
            start, end = match.span(0)
            entry[0] = (entry[0][start+1:end-1], entry[0][end:])
            entry[1] = 'localize "{}"'.format(entry[1])
            break
    print(entry)

files = glob(r'..\addons\**\*.sqf', recursive=True)
files += glob(r'..\addons\**\*.cpp', recursive=True)
files += glob(r'..\addons\**\*.hpp', recursive=True)
for file in files:
    path = os.path.normpath(file)
    addon = path.split(os.sep)[2].lower()
    #print(addon)
    with open(file, 'r') as stream:
        lines = stream.readlines()
    '''
    with open(file, 'w') as stream:
        for line in lines:
            for duplicate in duplicates:
                
            stream.write(line)
    '''

[('Attributes', 'Commanding'), 'localize "str_a3_commanding1"']
[('Camera', 'Camera'), 'localize "str_a3_cfgvehicles_land_camera_01_f0"']
[('Common', 'Enabled'), 'localize "str_a3_mp_groundsupport_modulemptypegroundsupport_enabled"']
[('Common', 'Disabled'), 'localize "str_a3_mp_groundsupport_modulemptypegroundsupport_disabled"']
[('Common', 'Yes'), 'localize "str_a3_cfgvehicles_modulestrategicmapimage_f_arguments_shadow_values_yes_0"']
[('Common', 'No'), 'localize "str_a3_cfgvehicles_modulestrategicmapimage_f_arguments_shadow_values_no_0"']
[('Common', 'Medium'), 'localize "STR_A3_Param_Fog_2"']
[('Common', 'Side'), 'localize "str_a3_rscdisplaydynamicgroups_side"']
[('Common', 'Sides'), 'localize "str_a3_mdl_category_systemsides"']
[('Common', 'Faction'), 'localize "str_a3_cfgvehicles_modulespawnai_f_arguments_faction0"']
[('Common', 'Aircraft'), 'localize "str_a3_wl_menu_aircraft"']
[('Common', 'Units'), 'localize "str_a3_rscdisplaycurator_modeunits_tooltip"']
[('Common', 'Vehicles')